In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle

     |████████████████████████████████| 61kB 7.6MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.12-cp37-none-any.whl size=73053 sha256=1c848d71ba800f3e518c24851a4b10550ad9e471d7a495704b6dd37e037b7f9e
  Stored in directory: /root/.cache/pip/wheels/a1/6a/26/d30b7499ff85a4a4593377a87ecf55f7d08af42f0de9b60303
Successfully built kaggle
  Found existing installation: kaggle 1.5.10
    Uninstalling kaggle-1.5.10:
      Successfully uninstalled kaggle-1.5.10


In [ ]:
import os 
os.mkdir('/content/kaggle')

In [ ]:
import json
token = {"username":"prabhatale","key":"016e28738b921e6c03f29cf3d1aba81c"}
with open('/content/kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

In [ ]:
!cp /content/kaggle/kaggle.json ~/.kaggle/kaggle.json

In [ ]:
!kaggle config set -n path -v{/content}

- path is now set to: {/content}


In [ ]:
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
import os
os.mkdir('/content/wheat_datasets')

In [ ]:
!kaggle competitions download -c global-wheat-detection -p /content/wheat_datasets

 98% 593M/607M [00:05<00:00, 121MB/s]
100% 607M/607M [00:05<00:00, 116MB/s]


In [ ]:
import zipfile
archive = zipfile.ZipFile('/content/wheat_datasets/global-wheat-detection.zip')
archive.extractall('/content/wheat_datasets')

In [ ]:
!git clone https://github.com/ultralytics/yolov5.git

Cloning into 'yolov5'...
remote: Enumerating objects: 5437, done.
remote: Total 5437 (delta 0), reused 0 (delta 0), pack-reused 5437
Receiving objects: 100% (5437/5437), 8.07 MiB | 37.04 MiB/s, done.
Resolving deltas: 100% (3713/3713), done.


In [ ]:
import pandas as pd
df = pd.read_csv('/content/wheat_datasets/train.csv')
df.head()

,image_id,width,height,bbox,source
0,b6ab77fd7,1024,1024,"[834.0, 222.0, 56.0, 36.0]",usask_1
1,b6ab77fd7,1024,1024,"[226.0, 548.0, 130.0, 58.0]",usask_1
2,b6ab77fd7,1024,1024,"[377.0, 504.0, 74.0, 160.0]",usask_1
3,b6ab77fd7,1024,1024,"[834.0, 95.0, 109.0, 107.0]",usask_1
4,b6ab77fd7,1024,1024,"[26.0, 144.0, 124.0, 117.0]",usask_1


In [ ]:
input_path = "/content/wheat_datasets"

In [ ]:
import ast
from tqdm import tqdm
import numpy as np
import shutil

In [ ]:
df.bbox = df.bbox.apply(ast.literal_eval)

In [ ]:
df = df.groupby("image_id")["bbox"].apply(list).reset_index(name="bboxes")

In [ ]:
df

,image_id,bboxes
0,00333207f,"[[0, 654, 37, 111], [0, 817, 135, 98], [0, 192..."
1,005b0d8bb,"[[765.0, 879.0, 116.0, 79.0], [84.0, 539.0, 15..."
2,006a994f7,"[[437.0, 988.0, 98.0, 36.0], [309.0, 527.0, 11..."
3,00764ad5d,"[[89.0, 256.0, 113.0, 107.0], [216.0, 282.0, 1..."
4,00b5fefed,"[[709.0, 97.0, 204.0, 105.0], [775.0, 250.0, 1..."
...,...,...
3368,ffb445410,"[[0.0, 534.0, 54.0, 118.0], [0.0, 480.0, 38.0,..."
3369,ffbf75e5b,"[[0, 697, 21, 58], [104, 750, 77, 75], [65, 84..."
3370,ffbfe7cc0,"[[256.0, 0.0, 64.0, 99.0], [390.0, 0.0, 48.0, ..."
3371,ffc870198,"[[447.0, 976.0, 78.0, 48.0], [18.0, 141.0, 218..."


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
df_train,df_valid = train_test_split(df,test_size=0.1,random_state=42,shuffle=True)

In [ ]:
df_train = df_train.reset_index(drop=True)

In [ ]:
df_valid = df_valid.reset_index(drop=True)

In [ ]:
import os
os.mkdir('/content/yolov5/wheat_data')
os.mkdir('/content/yolov5/wheat_data/images')
os.mkdir('/content/yolov5/wheat_data/labels')
os.mkdir('/content/yolov5/wheat_data/images/train')
os.mkdir('/content/yolov5/wheat_data/images/validation')
os.mkdir('/content/yolov5/wheat_data/labels/train')
os.mkdir('/content/yolov5/wheat_data/labels/validation')

In [ ]:
output_path = '/content/yolov5/wheat_data'
input_path = '/content/wheat_datasets'

In [ ]:
def process_data(df,split_type):
  for index,rows in tqdm(df.iterrows(),total=len(df)):
    image_name = rows["image_id"]
    bounding_boxes = rows["bboxes"]
    yolo_data = []
    for bbox in bounding_boxes:
      x = bbox[0]
      y = bbox[1]
      w = bbox[2]
      h = bbox[3]
      x_centre = (x + w)/2
      y_centre = (y + h)/2
      x_centre /= 1024
      y_centre /= 1024
      w /= 1024
      h /= 1024
      yolo_data.append([0,x_centre,y_centre,w,h])
    yolo_data = np.array(yolo_data)
    np.savetxt(os.path.join(output_path,f"labels/{split_type}/{image_name}.txt"),yolo_data,fmt=["%d","%f","%f","%f","%f"])
    shutil.copyfile(os.path.join(input_path,f"train/{image_name}.jpg"),
                os.path.join(output_path,f"images/{split_type}/{image_name}.jpg"))
    



In [ ]:
process_data(df=df_train,split_type="train")
process_data(df=df_valid,split_type="validation")

100%|██████████| 338/338 [00:00<00:00, 900.33it/s]


In [ ]:
cd yolov5

/content/yolov5


In [ ]:
!pip install -r requirements.txt

     |████████████████████████████████| 645kB 19.4MB/s 
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!pip install wandb

     |████████████████████████████████| 2.0MB 23.4MB/s 
     |████████████████████████████████| 102kB 14.0MB/s 
     |████████████████████████████████| 163kB 55.8MB/s 
     |████████████████████████████████| 133kB 55.1MB/s 
     |████████████████████████████████| 71kB 11.3MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8786 sha256=8422683c9f159a28bbf85c9e5152844dce5ce32612388db0414d4764fb7b1d89
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6489 sha256=7e16c244299689e048d67fc06159702572b74ba900d17f781f049c99bc77e76a
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
Successfully built pathtools subprocess32


In [ ]:
from google.colab import files
files.upload()

Saving wheat.yml to wheat.yml


{'wheat.yml': b'train: /content/yolov5/wheat_data/images/train\nval: /content/yolov5/wheat_data/images/validation\nnc: 1\nnames: ["Wheat"]'}

In [ ]:
!python train.py --img 1024 --batch 8 --epochs 5 --data wheat.yml --cfg models/yolov5l.yaml --name wheat_model

github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v4.0-138-ged2c742 torch 1.8.0+cu101 CUDA:0 (Tesla T4, 15109.75MB)

Namespace(adam=False, batch_size=8, bucket='', cache_images=False, cfg='models/yolov5l.yaml', data='wheat.yml', device='', entity=None, epochs=5, evolve=False, exist_ok=False, global_rank=-1, hyp='data/hyp.scratch.yaml', image_weights=False, img_size=[1024, 1024], linear_lr=False, local_rank=-1, log_artifacts=False, log_imgs=16, multi_scale=False, name='wheat_model', noautoanchor=False, nosave=False, notest=False, project='runs/train', quad=False, rect=False, resume=False, save_dir='runs/train/wheat_model3', single_cls=False, sync_bn=False, total_batch_size=8, weights='yolov5s.pt', workers=8, world_size=1)
Start Tensorboard with "tensorboard --logdir runs/train", view at http://localhost:6006/
2021-03-21 18:51:19.729452: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
hyperparame

In [ ]:
1